In [13]:
%pip install opencv-python mediapipe

Note: you may need to restart the kernel to use updated packages.


In [14]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose 

In [15]:
# Video Feed
# cap = cv2.VideoCapture(0)
# while cap.isOpened():
#     ret, frame = cap.read()
#     cv2.imshow('MediaPipe Feed', frame)

#     if cv2.waitKey(10) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()

In [16]:
# angle calculation fn

def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

In [17]:
cap = cv2.VideoCapture(0)
# mediapipe instance setup
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # here recoloring img
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # to make detections
        results = pose.process(image)

        # here recoloring back to original
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # landmark extraction
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except: 
            pass

        # detection rendering
        mp_drawing.draw_landmarks(image, results.pose_landmarks,  mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,280), thickness=2, circle_radius=2),
                                  )

        cv2.imshow('MediaPipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

[x: 0.6179891
y: 0.98016346
z: -1.4878293
visibility: 0.9995478
, x: 0.6479962
y: 0.9148396
z: -1.4012079
visibility: 0.99950504
, x: 0.66407156
y: 0.91257226
z: -1.4011666
visibility: 0.99959654
, x: 0.679179
y: 0.91106087
z: -1.4010576
visibility: 0.999524
, x: 0.5894842
y: 0.9214396
z: -1.3897959
visibility: 0.9994746
, x: 0.56732696
y: 0.9233265
z: -1.3899456
visibility: 0.9994747
, x: 0.54618543
y: 0.92635655
z: -1.3899211
visibility: 0.99936193
, x: 0.7046114
y: 0.92557454
z: -0.899984
visibility: 0.9995115
, x: 0.5159826
y: 0.9461779
z: -0.83667135
visibility: 0.9994324
, x: 0.65304255
y: 1.0283649
z: -1.2908556
visibility: 0.9927965
, x: 0.58244383
y: 1.0324758
z: -1.2752131
visibility: 0.99243647
, x: 0.87357914
y: 1.22169
z: -0.7446972
visibility: 0.35252538
, x: 0.42801085
y: 1.2534606
z: -0.5210853
visibility: 0.35678598
, x: 0.97681665
y: 1.2780429
z: -1.2560433
visibility: 0.13254263
, x: 0.3192112
y: 1.3216193
z: -0.83224475
visibility: 0.07830536
, x: 0.8271324
y: 1.121

In [18]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
shoulder

[0.9400768280029297, 0.8724195957183838]